# Análisis del riesgo de incumplimiento de los prestatarios

## Banco S - División de Préstamos

### Informe sobre la capacidad de prestatarios potenciales para pagar préstamos.

En este informe se desarrolla una **Puntuación de crédito**, métrica que evalúa la capacidad de prestatarios potenciales para pagar sus préstamos.

**Hipótesis**

1. El estado civil del prestatario influye en su capacidad de pago
2. El número de hijos del prestatario influye en su capacidad de pago

***

In [1]:
import pandas as pd
import numpy as np

In [2]:
credit = pd.read_csv('/datasets/credit_scoring_eng.csv')

***
> ## Exploración de los datos
***

In [3]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


>Se tienen 21525 observaciones, cuyas características son:

- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [4]:
# un vistazo general al dataset

credit.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


***
> **Observaciones**
>* La columna **days_employed** contiene números negativos. Se cambiará a **years_employed** ya que es una mejor métrica.
>* El nombre de la columna **dob_years** es ambiguo. Se cambiará a **age**, además que contiene muchos valores únicos que pueden categorizarse.
>* Dado el orden de magnitud de las columnas **days_employed** y **total_income**, éstas contienen decimales innecesarios.
>* La columna **education** contiene entradas que son la misma, pero escrita diferente (e.g., "secondary education", "SECONDARY EDUCATION", "Secondary Education").
>* Los valores de la columna **total_income** se cambiarán a miles de dólares. Además, se categorizará para un mejor análisis.
>* La columna **purchase** al parecer contiene muchos valores únicos que pueden categorizarse (e.g., "purchase of the house" y "purchase of the house for my family").
***

***
>Se procederá a aplicar lo mencionado en el apartado de observaciones, que son detalles mínimos para tener una tabla más limpia.

In [5]:
# renombrar las columnas
credit.rename(columns={'days_employed': 'years_employed', 'dob_years': 'age', 'total_income': 'total_income_(*1K)'},
              inplace=True)

# cambiar los valores negativos de la columna 'years_employed'
credit['years_employed'] = credit['years_employed'].map(lambda x: np.around(np.abs(x/365),decimals=1),
                                                        na_action='ignore')

# cambiar los valores de la columna 'total_income' a miles de dólares
credit['total_income_(*1K)'] = credit['total_income_(*1K)'].map(lambda x: int(x) / 1000,
                                                        na_action='ignore')

# reformatear todos los nombres de la columna 'education'
credit['education'] = credit['education'].str.lower().str.replace('degree', '').str.replace('education', '')
credit['education'] = credit['education'].str.replace("bachelor's", 'bachelor').str.strip()

In [6]:
# definir funciones que categorizarán los valores de varias columnas

def purpose_mapper(text):
    """
    Se seleccionan palabras clave en la columna 'purpose'
    Cada propósito cae dentro de alguna de las categorías mostradas en category_dict
    """
    category_dict = {
        'house': 0,
        'wedding': 1,
        'education': 2, 'university': 2, 'educated': 2,
        'estate': 3,
        'building': 4,
        'car': 5, 'cars': 5,
        'property': 6,
        'housing': 7,
    }
    
    for word in text.split():
        if word in category_dict.keys():
            return category_dict[word]
    
    
def age_mapper(age):
    """
    Se seleccionan estas categorías como representativas de la población
    """
    
    if 19 <= age <= 25:
        return 0
    if 25 < age <= 30:
        return 1
    if 30 < age <= 35:
        return 2
    if 35 < age <= 40:
        return 3
    if 40 < age <= 45:
        return 4
    if 45 < age <= 50:
        return 5
    if 50 < age <= 55:
        return 6
    if 55 < age <= 60:
        return 7
    if 60 < age:
            return 8
    return age
    
def total_income_mapper(income): # de acuerdo a salaryexplorer.com
    """
    Se utilizó esta categorización basada en salaryexplorer.com
    donde se muestra información de los salarios promedios
    en los EEUU
    """
    if income <= 37.6:
        return 'low'
    if 37.6 < income <= 168:
        return 'average'
    
    return 'high' 

# ----

# creación de la columna 'purpose_id', que categoriza la columna 'purpose'
credit['purpose_id'] = credit['purpose'].map(purpose_mapper)

# creación de la columna 'age_id', que categoriza la columna 'age'
age_id_column = credit['age'].map(age_mapper)
credit.insert(3, 'age_id', age_id_column)

# creación de la columna 'total_income_id', que categoriza la columna 'total_income'
total_income_id_column = credit['total_income_(*1K)'].map(total_income_mapper)
credit.insert(12, 'total_income_id', total_income_id_column)

# muestra la información del dataset

credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21525 non-null  int64  
 1   years_employed      19351 non-null  float64
 2   age                 21525 non-null  int64  
 3   age_id              21525 non-null  int64  
 4   education           21525 non-null  object 
 5   education_id        21525 non-null  int64  
 6   family_status       21525 non-null  object 
 7   family_status_id    21525 non-null  int64  
 8   gender              21525 non-null  object 
 9   income_type         21525 non-null  object 
 10  debt                21525 non-null  int64  
 11  total_income_(*1K)  19351 non-null  float64
 12  total_income_id     21525 non-null  object 
 13  purpose             21525 non-null  object 
 14  purpose_id          21525 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 2.5+

***
> Se observa que las columnas **years_employed** y **total_income_(*1K)** son las únicas que contienen valores ausentes. Se analizará más a fondo para visualizar cómo abordarlos.
***

In [7]:
# creación de un subconjunto del dataset original, que contiene sólo las filas con datos ausentes

credit_only_nan = credit[(credit['years_employed'].isnull()) & credit['total_income_(*1K)'].isnull()]
credit_only_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            2174 non-null   int64  
 1   years_employed      0 non-null      float64
 2   age                 2174 non-null   int64  
 3   age_id              2174 non-null   int64  
 4   education           2174 non-null   object 
 5   education_id        2174 non-null   int64  
 6   family_status       2174 non-null   object 
 7   family_status_id    2174 non-null   int64  
 8   gender              2174 non-null   object 
 9   income_type         2174 non-null   object 
 10  debt                2174 non-null   int64  
 11  total_income_(*1K)  0 non-null      float64
 12  total_income_id     2174 non-null   object 
 13  purpose             2174 non-null   object 
 14  purpose_id          2174 non-null   int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 271.

***
>Se observa que las filas en las que **years_employed** tiene valores ausentes, son las mismas filas en las que **total_income** tiene valores ausentes. Esto sugiere la posible existencia de un patrón. Primero, se analizará cuanto porcentaje del dataset original representa el subset de las observaciones con valores ausentes.
***

In [8]:
nan_ratio = len(credit_only_nan) / len(credit)
print(f'El porcentaje de observaciones con valores ausentes es del {nan_ratio:.1%}')

El porcentaje de observaciones con valores ausentes es del 10.1%


***
>Dado que el porcentaje no es tan significativo, se procederá a analizar si existe algún patrón en las observaciones con los valores ausentes para decidir cómo abordarlos.
***

In [9]:
# realizar un conteo sobre la columna 'children'

credit_only_nan['children'].value_counts()

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

***
>La mayoría de observaciones en este subconjunto del dataset original, pertenece a usuarios que no tienen hijos.
***

In [10]:
# realizar un conteo sobre la columna 'education'

credit_only_nan['education'].value_counts()

secondary       1540
bachelor         544
some college      69
primary           21
Name: education, dtype: int64

***
>El factor educación parece ser más confiable. Si la educación del usuario es Secundaria, tal vez eso explica que no haya contastado las cuestiones sobre su antigüedad laboral y su salario, pues tal vez su trabajo no es muy estable, y su salario no muy alto.
***

In [11]:
# realizar un conteo sobre la columna 'age_id'

credit_only_nan['age_id'].value_counts()

3    287
2    286
4    268
5    254
6    250
7    239
1    236
8    223
0    131
Name: age_id, dtype: int64

***
>La mayoría de los usuarios se encuentran en un rango de edad entre los 26 y 50 años. Eso es de esperarse, ya que los menores de edad y los adultos mayores no suelen pedir prestamos tan frecuentemente. Esta característica no parece ser relevante para encontran el patrón que buscamos.
***

In [12]:
# realizar un conteo sobre la columna 'family_status_id'

credit_only_nan['family_status_id'].value_counts()

0    1237
1     442
4     288
3     112
2      95
Name: family_status_id, dtype: int64

***
>La mayoría de los usuarios son casados. Esto puede influir en la falta de respuestas a las features de interés.
***

In [13]:
# realizar un conteo sobre la columna 'gender'

credit_only_nan['gender'].value_counts()

F    1484
M     690
Name: gender, dtype: int64

***
>La mayoría de los usuarios son mujeres.
***

In [14]:
# realizar un conteo sobre la columna 'income_type'

credit_only_nan['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

***
>La mayoría de los usuarios son empleados.
***

In [15]:
# realizar un conteo sobre la columna 'debt'

credit_only_nan['debt'].value_counts()

0    2004
1     170
Name: debt, dtype: int64

***
>La mayoría de los usuarios no tienen una deuda.
***

In [16]:
# realizar un conteo sobre la columna 'purpose_id'

credit_only_nan['purpose_id'].value_counts()

2    425
5    418
3    402
1    249
7    204
6    202
0    169
4    105
Name: purpose_id, dtype: int64

***
>Los 3 primeros lugares de esta característica, pertenecen a usuarios que quieren invertir en su real estate, su educación, o en un auto. Sin embargo, no se observan diferencias significativas.
***

>**Conclusiones intermedias**

>Las observaciones que contienen valores ausentes en las características **years_employed** y **total_income** pertenecen en su mayoría a personas que:
>* Tienen sólo Educación Secundaria
>* Son casadas
>* No tienen hijos
>* Son mujeres
>* Tienen empleo
>* Tienen entre 26 y 50 años
>* No tienen aduedo

> Estas conclusiones se tomarán en cuenta para reemplazar los valores ausentes. Pero primero, se inspeccionarán las columnas para encontrar irregularidades.

> ### Análisis de las columnas del dataset

> #### Columna 'children'

In [17]:
credit['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

***
>Al parecer hubo un error al registrar usuarios con 20 y -1 hijos. La cantidad de estos errores es mínima, por lo que se substituirá 20 por 2, y -1 por 1, dado que parece un simple error humano.
***

In [18]:
credit['children'].replace({20:2, -1:1}, inplace=True)
credit['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

> #### Columna 'years_employed'

In [19]:
credit['years_employed'].value_counts()

0.6       297
0.5       257
1.2       231
0.7       231
0.3       226
         ... 
988.9       1
1090.1      1
951.4       1
1029.2      1
943.6       1
Name: years_employed, Length: 2024, dtype: int64

***
>Al parecer existen varios errores al considerar valores muy atípicos para esta columna. Se procederá a investigar más a fondo.
***

In [20]:
# analizar cuantos usuarios aparecen con más de 45 años de empleo.


credit[(credit['years_employed'] > 45)]

,children,years_employed,age,age_id,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_(*1K),total_income_id,purpose,purpose_id
4,0,932.2,53,6,secondary,1,civil partnership,1,F,retiree,0,25.378,low,to have a wedding,1
18,0,1096.7,53,6,secondary,1,widow / widower,2,F,retiree,0,9.091,low,buying a second-hand car,5
24,1,927.5,57,7,secondary,1,unmarried,4,F,retiree,0,46.487,average,transactions with commercial real estate,3
25,0,996.0,67,8,secondary,1,married,0,M,retiree,0,8.818,low,buy real estate,3
30,1,919.4,62,8,secondary,1,married,0,F,retiree,0,27.432,low,transactions with commercial real estate,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,928.5,53,6,secondary,1,civil partnership,1,M,retiree,0,12.070,low,to have a wedding,1
21508,0,1058.9,62,8,secondary,1,married,0,M,retiree,0,11.622,low,property,6
21509,0,992.2,59,7,bachelor,0,married,0,M,retiree,0,11.684,low,real estate transactions,3
21518,0,1024.6,59,7,secondary,1,married,0,F,retiree,0,24.618,low,purchase of a car,5


***
>La cifra de observaciones con años totales trabajados mayores a 45 es alta. A simple vista, se observa que los usuarios parece que son retirados (retiree). Se analizará más a fondo.
***

In [21]:
len(credit[(credit['years_employed'] > 45) & (credit['income_type'] == 'retiree')])

3443

***
>De 3448 observaciones con valores de años trabajados atípicos, 3443 son retirados. Esto representa el 16% de las observaciones totales, lo cual es un porcentaje considerable con lo que se procederá a reemplazar esos valores atípicos.
***

***
>Para dicha tarea, lo ideal sería contar con información acerca de los usuarios retirados ('retiree'), para obtener la media de sus años laborados. Sin embargo, como se muestra a continuación, todos los retirados tienen valores atípicos, como se muestra a continuación.
***

In [22]:
# demostración de que no hay filas de usuarios retirados y que tengan menos de 100 años laborando

len(credit[(credit['income_type'] == 'retiree') & (credit['years_employed'] < 100)])

0

>Debido a esto, se analizarán los datos de los demás usuarios con valores atípicos que no son 'retiree'.

In [23]:
credit[(credit['years_employed'] > 45) & (credit['income_type'] != 'retiree')]

,children,years_employed,age,age_id,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_(*1K),total_income_id,purpose,purpose_id
3133,1,924.7,31,2,secondary,1,married,0,M,unemployed,1,9.593,low,buying property for renting out,6
4299,0,48.3,61,8,secondary,1,married,0,F,business,0,19.609,low,purchase of the house,0
7329,0,45.5,60,7,bachelor,0,married,0,F,employee,0,19.951,low,going to university,2
14798,0,1083.0,45,4,bachelor,0,civil partnership,1,F,unemployed,0,32.435,low,housing renovation,7
16335,1,50.4,61,8,secondary,1,married,0,F,employee,0,29.788,low,real estate transactions,3


***
>Descubrimos que en este subconjunto, hay 3 usuarios mayores de 60 años que tienen valores no atípicos para años laborados. Esto es, usuarios en edad de retiro cuyos años laborados podemos utilizar para reemplazar los datos de los 'retiree'. Notamos que la media en este caso es 48.1

>Sin embargo, notemos que existe una entrada para un usuario de 31 años que tiene un valor atípico para los años laborados. En este caso, se utilizarán los datos de los otros usuarios de esa edad y con educación secundaria. Se calculará la media y se utilizará para reemplazar el valor atípico mencionado.
***

In [24]:
# obtener los datos filtrados

aux_table = credit[(credit['age'] == 31) & (credit['education_id'] == 1)]

# calcular la media

mean_31_years_old = np.around(aux_table['years_employed'].mean(), decimals=1)

# reemplazar el valor atípico por la media

credit.at[3133, 'years_employed'] = mean_31_years_old
credit.iloc[3133]

children                                            1
years_employed                                    7.6
age                                                31
age_id                                              2
education                                   secondary
education_id                                        1
family_status                                 married
family_status_id                                    0
gender                                              M
income_type                                unemployed
debt                                                1
total_income_(*1K)                              9.593
total_income_id                                   low
purpose               buying property for renting out
purpose_id                                          6
Name: 3133, dtype: object

***
>Ahora, se reemplazarán los valores atípicos restantes.

>Primero, se investigará el rango de edades que se tiene para los usuarios con valores atípicos. En base a esas edades, se calcularán las medias de los usuarios que no tienen valores atípicos, para reemplazar los valores atípicos.
***

In [25]:
credit[credit['years_employed'] > 45]['age'].value_counts()

59    254
60    244
62    235
61    216
57    212
58    208
63    192
56    184
64    179
55    162
54    145
66    139
65    136
67    132
53    105
52     95
68     80
69     74
51     73
50     61
70     54
71     48
49     30
72     28
48     20
0      17
46     13
47     13
45     11
44     10
42      9
43      9
38      8
40      7
73      6
41      6
37      5
74      4
39      4
36      3
34      3
32      3
27      3
33      2
26      2
22      1
35      1
28      1
Name: age, dtype: int64

***
>Descubrimos ahora que existen usuarios con edades de retiro atípicas también. Investiguemos un poco más.
***

In [26]:
credit[(credit['years_employed'] > 45) & (credit['age'] <= 50)]

,children,years_employed,age,age_id,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_(*1K),total_income_id,purpose,purpose_id
99,0,949.4,0,0,secondary,1,married,0,F,retiree,0,11.406,low,car,5
157,0,954.6,38,3,secondary,1,married,0,F,retiree,1,18.169,low,purchase of a car,5
311,0,970.5,44,4,secondary,1,married,0,F,retiree,0,7.593,low,going to university,2
388,0,991.6,49,5,bachelor,0,unmarried,4,F,retiree,0,46.431,average,cars,5
512,1,1087.7,46,5,secondary,1,married,0,F,retiree,0,30.672,low,housing transactions,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20904,3,921.5,42,4,secondary,1,married,0,M,retiree,0,21.233,low,to own a car,5
21072,0,1005.2,50,5,secondary,1,married,0,F,retiree,1,17.297,low,buy commercial real estate,3
21104,0,1007.0,47,5,primary,3,civil partnership,1,F,retiree,0,9.609,low,buying my own car,5
21168,0,942.7,49,5,secondary,1,married,0,F,retiree,0,23.419,low,buy residential real estate,3


***
>Al observar la tabla resutante, podemos ver que es posible obtener valores sensatos para los años laborados si reducimos en 2 el orden de magnitud de esa columna. Mediante este approach, tendríamos que modificar la columna 'income_type' para que también sea congruente con las edades de los usuarios.

> El approach que tomaremos será eliminar estas filas ya que son solamente 242 y no afectarán de manera significativa el anális subsecuente.
***

In [27]:
# eliminación de las filas problemáticas
# y demostración de que se han eliminado

indexes_to_delete_years = credit[(credit['years_employed'] > 45) & (credit['age'] <= 50)].index
credit.drop(index=indexes_to_delete_years, inplace=True)
len(credit[(credit['years_employed'] > 45) & (credit['age'] <= 50)])

0

In [28]:
# distribución de usuarios retirados y con valores atípicos

credit[credit['years_employed'] > 45]['age'].value_counts()

59    254
60    244
62    235
61    216
57    212
58    208
63    192
56    184
64    179
55    162
54    145
66    139
65    136
67    132
53    105
52     95
68     80
69     74
51     73
70     54
71     48
72     28
73      6
74      4
Name: age, dtype: int64

***
>Calcularemos la media y la mediana para años laborados de los usuarios sin valores atípicos. Se agruparán de acuerdo a 'age_id.

In [29]:
# cálculo de las medias por grupo de edad

grouped_by_age_id = credit.groupby('age_id')['years_employed'].agg(['mean', 'median'])
grouped_by_age_id

,mean,median
age_id,,
0,2.750506,2.30
1,3.987717,3.40
2,5.146817,4.10
3,6.313839,4.90
4,7.292116,5.40
5,8.170763,5.90
6,286.658119,10.10
7,557.481407,922.45
8,796.462806,975.90


***
>Se utilizará la media para los id de 0 a 5, mientras que se utilizará la mediana para el id 6. Para los id 7 y 8, se utilizará 48.1, que es la media de los usuarios retirados que no presentan valores atípicos, como se mencionó anteriormente.
***

In [30]:
# definir una función que tome una fila y reemplace los valores donde sea necesario

def replace_outliers(row): 
    years = row['years_employed']
    age_id = row['age_id']

    if years > 100:
        try:
            if age_id == 6:
                return grouped_by_age_id.loc[age_id, 'median']
            if (age_id == 7) or (age_id == 8):
                return 48.1
            return grouped_by_age_id.loc[age_id, 'mean']
        except:
            return years
    return years

# reemplazo de los valores atípicos

credit['years_employed'] = credit.apply(replace_outliers, axis=1)

In [31]:
# comprobando el valor máximo de 'years_employed'

credit['years_employed'].max()

50.4

> #### Columna 'age'

In [32]:
np.sort(credit['age'].unique())

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

In [33]:
# análisis de cuántos usuarios aparecen con 0 años de edad

len(credit[credit['age'] == 0])

84

***
>84 filas contienen un valor de edad de 0 años. Se observa también que hay muchos valores diferentes en las demás columnas, i.e., no se ve un patrón claro, con lo que éstas filas se eliminarán.
***

In [34]:
# eliminación de las filas problemáticas
# y demostración de que se han eliminado

indexes_to_delete_age = credit[credit['age'] == 0].index
credit.drop(index=indexes_to_delete_age, inplace=True)
credit[credit['age'] == 0]

,children,years_employed,age,age_id,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_(*1K),total_income_id,purpose,purpose_id


> #### Columna 'education'

In [35]:
credit['education'].value_counts()

secondary       14991
bachelor         5186
some college      738
primary           278
graduate            6
Name: education, dtype: int64

***
>No se realizarán cambios en la columna 'education', pues todo se ve en orden
***

> #### Columna 'family_status'

In [36]:
credit['family_status'].value_counts()

married              12196
civil partnership     4108
unmarried             2775
divorced              1172
widow / widower        948
Name: family_status, dtype: int64

***
>No se realizarán cambios en la columna 'family_status' pues todo se ve en orden
***

> #### Columna 'gender'

In [37]:
credit['gender'].value_counts()

F      14022
M       7176
XNA        1
Name: gender, dtype: int64

***
>Sólo existe una columna con un valor indefinido. Se cambiará simplemente por 'undefined' para mayor claridad.
***

In [38]:
credit.loc[credit['gender'] == 'XNA', 'gender'] = 'undefined'
credit['gender'].value_counts()

F            14022
M             7176
undefined        1
Name: gender, dtype: int64

> #### Columna 'debt'

In [39]:
credit['debt'].value_counts()

0    19483
1     1716
Name: debt, dtype: int64

***
>No hay ningún problema significativo en esta columna.
***

> #### Columna 'total_income'

***
>Se analizarán valores atípicos. Los valores ausentes de tratarán más adelante.
***

In [40]:
min_income = credit['total_income_(*1K)'].min()
max_income = credit['total_income_(*1K)'].max()

print(f'Ingreso mínimo en el dataset: {min_income*1000}')
print(f'Ingreso máximo en el dataset: {max_income*1000}')

Ingreso mínimo en el dataset: 3306.0
Ingreso máximo en el dataset: 362496.0


***
>Se tiene un rango amplio de valores en los sueldos. Esto se tendrá en cuenta cuando se vayan a reemplazar los valores ausentes.
***
***

***
>Una vez analizadas las columnas y tratados los problemas en cada una, se reiniciarán los índices del dataset para tener una tabla limpia.
***

In [41]:
credit.reset_index(drop=True, inplace=True)
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21199 entries, 0 to 21198
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21199 non-null  int64  
 1   years_employed      19035 non-null  float64
 2   age                 21199 non-null  int64  
 3   age_id              21199 non-null  int64  
 4   education           21199 non-null  object 
 5   education_id        21199 non-null  int64  
 6   family_status       21199 non-null  object 
 7   family_status_id    21199 non-null  int64  
 8   gender              21199 non-null  object 
 9   income_type         21199 non-null  object 
 10  debt                21199 non-null  int64  
 11  total_income_(*1K)  19035 non-null  float64
 12  total_income_id     21199 non-null  object 
 13  purpose             21199 non-null  object 
 14  purpose_id          21199 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 2.4+

***
>## Reemplazo de valores ausentes
***

> Cómo se explicó anteriormente, las columnas **years_employed** y **total_income** poseen valores ausentes en las mismas filas. Al reemplazar los valores, no se tendrán en cuenta la una sobre la otra, sino las columnas restantes.

> ### Columna 'years_employed'
> Para esta columna, se tendrán en cuenta las conclusiones obtenidas durante el análisis de los valores ausentes. Particularmente, se tomarán en cuenta la edad y el tipo de trabajo. Esto debido a que éstas características arrojaron la mayoría de las filas con valores ausentes (Consultar las **Conclusiones intermedias** en la sección '1 Exploración de los datos').

>Se utilizará la edad para filtrar los datos, ya que la edad es el factor más importante para determinar los años de servicio.

In [42]:
years_employed_pivot_table = credit.pivot_table(
                            index=['age_id', 'income_type'],
                               values='years_employed',
                               aggfunc=['median', 'mean', 'std'])

# tabla auxiliar que sirve por si un usuario no cumple con las características age_id + income_type
# por ejemplo, un usuario age_id=0 + income_type=retiree
# se utilizará sólo la edad en este caso para reemplazar el valor

years_employed_aux_pivot_table = credit.pivot_table(
                            index=['age_id'],
                               values='years_employed',
                               aggfunc=['median', 'mean', 'std'])


print(years_employed_aux_pivot_table)
years_employed_pivot_table

               median           mean            std
       years_employed years_employed years_employed
age_id                                             
0                 2.2       2.532284       1.820463
1                 3.4       3.987717       2.822945
2                 4.1       5.146817       4.004544
3                 4.9       6.313839       5.170365
4                 5.4       7.292116       6.319557
5                 5.9       8.170763       7.360187
6                10.1       9.174067       7.108760
7                48.1      30.855025      20.174126
8                48.1      40.354356      15.868297


median           mean  \
                                   years_employed years_employed   
age_id income_type                                                 
0      business                              2.10       2.371182   
       civil servant                         3.10       3.219697   
       employee                              2.20       2.548711   
       student                               1.60       1.600000   
1      business                              3.10       3.803636   
       civil servant                         4.90       5.140838   
       employee                              3.30       3.918781   
       entrepreneur                          1.40       1.400000   
2      business                              3.85       4.802049   
       civil servant                         6.10       6.464824   
       employee                              4.00       5.139294   
       unemployed                            7.60       7.600000   
3      business                              4.50       5.605518   
       civil servant                         9.35       9.684314   
       employee                              4.70       6.211988   
       paternity / maternity leave           9.00       9.000000   
4      business                              4.90       6.323300   
       civil servant                         8.80      10.572897   
       employee                              5.10       7.270966   
5      business                              5.60       7.496333   
       civil servant                         9.65      11.589130   
       employee                              5.70       8.003358   
6      business                              5.40       7.869617   
       civil servant                         9.80      13.014634   
       employee                              6.10       8.681316   
       retiree                              10.10      10.100000   
7      business                              5.90       8.077143   
       civil servant                        11.50      14.370886   
       employee                              6.10       9.432212   
       retiree                              48.10      48.100000   
8      business                              6.50      10.115447   
       civil servant                         8.80      11.652174   
       employee                              7.30      10.656388   
       retiree                              48.10      48.100000   

                                              std  
                                   years_employed  
age_id income_type                                 
0      business                          1.701794  
       civil servant                     2.252876  
       employee                          1.820000  
       student                                NaN  
1      business                          2.761867  
       civil servant                     3.256832  
       employee                          2.753197  
       entrepreneur                           NaN  
2      business                          3.740745  
       civil servant                     4.235155  
       employee                          4.059389  
       unemployed                             NaN  
3      business                          4.545400  
       civil servant                     5.961382  
       employee                          5.175410  
       paternity / maternity leave            NaN  
4      business                          5.237916  
       civil servant                     7.396415  
       employee                          6.451929  
5      business                          6.643587  
       civil servant                     8.265359  
       employee                          7.414299  
6      business                          7.668248  
       civil servant                    10.056494  
       employee                          8.252904  
       retiree                           0.000000  
7      business   

> Se tulizará la media para cada grupo de edades y subgrupo de tipo de empleo para el reemplazo de los valores ausentes.

In [43]:
# definir una función que reemplazará los valores ausentes

def fill_null_years(row):
    age_id = row['age_id']
    income_type = row['income_type'] 
    years = row['years_employed']

    if pd.isna(years):
        try:
            return years_employed_pivot_table.loc[(age_id,income_type),('mean','years_employed')]
        except:
            return years_employed_aux_pivot_table.loc[(age_id),('mean','years_employed')]
    return years

# reemplazar los valores ausentes

credit['years_employed'] = credit.apply(fill_null_years, axis=1)

# mostrar la información del dataset

credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21199 entries, 0 to 21198
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21199 non-null  int64  
 1   years_employed      21199 non-null  float64
 2   age                 21199 non-null  int64  
 3   age_id              21199 non-null  int64  
 4   education           21199 non-null  object 
 5   education_id        21199 non-null  int64  
 6   family_status       21199 non-null  object 
 7   family_status_id    21199 non-null  int64  
 8   gender              21199 non-null  object 
 9   income_type         21199 non-null  object 
 10  debt                21199 non-null  int64  
 11  total_income_(*1K)  19035 non-null  float64
 12  total_income_id     21199 non-null  object 
 13  purpose             21199 non-null  object 
 14  purpose_id          21199 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 2.4+

> ### Columna 'total_income_(*1K)'
> Para esta columna, también se tendrán en cuenta las conclusiones obtenidas durante el análisis de los valores ausentes. Particularmente, se tomarán en cuenta el género, la educación, y el número de hijos. Esto debido a que éstas características arrojaron la mayoría de las filas con valores ausentes (Consultar las **Conclusiones intermedias** en la sección '1 Exploración de los datos').

>Principalmente se utilizará el nivel educativo para filtrar los datos, ya que ese es uno de los factores más importantes para medir el ingreso.

In [44]:
total_income_pivot_table = credit.pivot_table(
                            index=['education', 'income_type'],
                               values='total_income_(*1K)',
                               aggfunc=['median', 'mean', 'std'])

total_income_pivot_table

median  \
                                         total_income_(*1K)   
education    income_type                                      
bachelor     business                               32.2650   
             civil servant                          27.6015   
             employee                               26.5350   
             entrepreneur                           79.8660   
             retiree                                22.7955   
             student                                15.7120   
graduate     civil servant                          17.8220   
             employee                               31.7710   
             retiree                                28.3340   
primary      business                               21.8870   
             civil servant                          23.7335   
             employee                               20.1590   
             retiree                                16.4150   
secondary    business                               25.4460   
             civil servant                          21.9120   
             employee                               21.8530   
             paternity / maternity leave             8.6120   
             retiree                                18.2715   
             unemployed                              9.5930   
some college business                               28.7780   
             civil servant                          25.6940   
             employee                               24.2090   
             retiree                                19.6330   

                                                       mean                std  
                                         total_income_(*1K) total_income_(*1K)  
education    income_type                                                        
bachelor     business                             38.815121          26.741683  
             civil servant                        31.617434          18.297123  
             employee                             30.672077          18.054728  
             entrepreneur                         79.866000                NaN  
             retiree                              26.453243          16.343666  
             student                              15.712000                NaN  
graduate     civil servant                        17.822000                NaN  
             employee                             31.089000          12.211292  
             retiree                              28.334000          17.725753  
primary      business                             26.408621          11.915970  
             civil servant                        29.448333          25.724289  
             employee                             21.953556          10.186515  
             retiree                              17.910978           9.123071  
secondary    business                             28.731534          15.846106  
             civil servant                        24.663802          12.817864  
             employee                             24.427669          13.247323  
             paternity / maternity leave           8.612000                NaN  
             retiree                              20.922932          11.594741  
             unemployed                            9.593000                NaN  
some college business                             31.623379          17.201711  
             civil servant                        27.595848          14.027817  
             employee                             27.934157          14.487298  
             retiree                              23.491774          12.570670

***
>Dado que se observan unas desviaciones standard altas, se utilizará la mediana para reemplazar los valores ausentes de esta columna. En el caso de Graduate education, la media y la mediana son la misma, cabe mencionar que en este caso no se tienen valores de mujeres con 0 hijos, sino mujeres con 3 hijos. Se utilizará ese valor.
***

In [45]:
# definir una función que reemplazará los valores ausentes

def fill_null_income(row):
    education = row['education']
    income_type = row['income_type'] 
    total_income = row['total_income_(*1K)']

    if pd.isna(total_income):
        try:
            return total_income_pivot_table.loc[(education,income_type),('mean','total_income_(*1K)')]
        except:
            return total_income
    return total_income

# reemplazar los valores ausentes

credit['total_income_(*1K)'] = credit.apply(fill_null_income, axis=1)

# mostrar la información del dataset

credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21199 entries, 0 to 21198
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21199 non-null  int64  
 1   years_employed      21199 non-null  float64
 2   age                 21199 non-null  int64  
 3   age_id              21199 non-null  int64  
 4   education           21199 non-null  object 
 5   education_id        21199 non-null  int64  
 6   family_status       21199 non-null  object 
 7   family_status_id    21199 non-null  int64  
 8   gender              21199 non-null  object 
 9   income_type         21199 non-null  object 
 10  debt                21199 non-null  int64  
 11  total_income_(*1K)  21199 non-null  float64
 12  total_income_id     21199 non-null  object 
 13  purpose             21199 non-null  object 
 14  purpose_id          21199 non-null  int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 2.4+

***
>## Comprobación de las hipótesis
***

**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [46]:
children_debt = credit.pivot_table(index='children', values='debt', aggfunc= ['count','sum'])
children_debt['debt_rate'] = (children_debt[('sum','debt')] / children_debt[('count','debt')])*100
children_debt

,count,sum,debt_rate
,debt,debt,
children,,,
0,13914,1047,7.524795
1,4802,438,9.121200
2,2110,201,9.526066
3,324,26,8.024691
4,40,4,10.000000
5,9,0,0.000000


>**Conclusión**

>Los resultados muestran que efectivamente hay relación entre el número de hijos y el incumplimiento en el pago. Esto confirma la hipótesis 2.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [47]:
status_debt = credit.pivot_table(index='family_status', values='debt', aggfunc= ['count','sum'])
status_debt['debt_rate'] = (status_debt[('sum','debt')] / status_debt[('count','debt')])*100
status_debt

,count,sum,debt_rate
,debt,debt,
family_status,,,
civil partnership,4108,380,9.250243
divorced,1172,84,7.167235
married,12196,918,7.527058
unmarried,2775,272,9.801802
widow / widower,948,62,6.540084


>**Conclusión**

>Existe una relación entre el estado civil y el incumplimiento en los pagos. Las personas que tienen pareja, los solteros, los divorciados y los casados tienden a incumplir en sus pagos. Esto confirma la hipótesis 1.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [48]:
income_debt = credit.pivot_table(index='total_income_id', values='debt', aggfunc= ['count','sum'])
income_debt['debt_rate'] = (income_debt[('sum','debt')] / income_debt[('count','debt')])*100
income_debt

,count,sum,debt_rate
,debt,debt,
total_income_id,,,
average,3219,223,6.927617
high,2185,171,7.826087
low,15795,1322,8.369737


>**Conclusión**

>Se observa que los usuarios de ingresos bajos tienden más a incumplir en sus pagos.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [49]:
purpose_debt = credit.pivot_table(index='purpose_id', values='debt', aggfunc= ['count','sum'])
purpose_debt['debt_rate'] = (purpose_debt[('sum','debt')] / purpose_debt[('count','debt')])*100
print(purpose_debt)

category_dict = {
        'house': 0,
        'wedding': 1,
        'education': 2, 'university': 2, 'educated': 2,
        'estate': 3,
        'building': 4,
        'car': 5, 'cars': 5,
        'property': 6,
        'housing': 7,
    }
print()
for purpose, num in category_dict.items():
    print(f'{num}: {purpose}')

           count  sum debt_rate
            debt debt          
purpose_id                     
0           1882  125  6.641870
1           2307  179  7.758994
2           3962  367  9.262998
3           3793  285  7.513841
4           1225  102  8.326531
5           4259  397  9.321437
6           1891  133  7.033316
7           1880  128  6.808511

0: house
1: wedding
2: education
2: university
2: educated
3: estate
4: building
5: car
5: cars
6: property
7: housing


>**Conclusión**

>Los usuarios más propensos a endeudarse son aquellos que piden un préstamo para un auto, seguidos de aquellos que quieren invertir en su educación, y quienes quieren el dinero para invertir en una casa.

# Conclusión general

>El conjunto de datos original contenía numerosas observaciones con datos ausentes. Tal falta de datos se observó en las columnas de años trabajados y en los ingresos totales del usuario. Se encontraron ciertos patrones en las observaciones, como que los usuarios en su mayoría tenía un nivel de educación secundaria y eran mujeres.

>Además, los años trabajados para algunos usuarios retirados mostraban valores muy poco convencionales. Se decidió reemplazar esos valores por algunos representativos de los usuarios en base a sus edades, ya que la edad es un factor determinante para calcular el número de años laborados.

>Se eliminaron filas con datos como edad igual a 0 años, y algunos con edades de retiro atípicas. Tal eliminación se basó en que las filas representaban un porcentaje ínfimo del dataset, lo que no influye dramáticamente en el análisis realizado.

>Las conclusiones a las que se arribaron muestran que los usuarios que:
>* tienen hijos
>* tienen pareja, son solteros o divorciados
>* ganan menos de 38 mil dólares anuales
>* quieren el préstamo para invertirlo en un auto, en educación, o en una casa

>son más propensos a no pagar a tiempo un préstamo.